In [3]:
from langchain.tools import tool
from langchain import hub

from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

In [5]:
@tool
def get_columns(file_name: str) -> list:
    """Get columns from xlsx file. Output list of column name"""
    import pandas as pd

    dataframe = pd.read_excel(file_name)
    return dataframe.columns.tolist()

@tool
def generate_cleaning_code(actions:str, cols:list) -> str:
    """Generate cleaning code using open ai. Input is the actions to perform and the available list of columns in the excel file. Output is a string of code."""
        
    openai = ChatOpenAI( model="gpt-4-0125-preview",api_key="sk-", temperature = 0)
    template = """The excel has the following columns {cols}.

    Write a react code to do the following cleaning operations:

    {question}

    Use the following code template:

    ```typescript
    {code}
        ```

    make sure to convert the types of item to the appropriate type before performing any operations on them.
    Also handle any null values with ? operator.
        
    Provide only the code without any explanations.     
    """


    prompt = PromptTemplate.from_template(template)
    prompt = prompt.partial(code = """ <NuvoImporter
    licenseKey="Your License Key"
    settings={\{
        developerMode: true,
        identifier: "customer_data",
        columns: [
        {
            label: "Customer Code",
            key: "customer_code",
            columnType: "string",
        },
        {
            label: "Phone Number",
            key: "phone_number",
            columnType: "string",
        },
        ],
    }}
    columnHooks={{
        phoneNumber: (values) => {
        values.map(([item, index]) => {
            let phoneNumber = item;
            if (/^[0]{{2}}/.test(phoneNumber)) {
            phoneNumber = `+${item.slice(2, item.length)}`;
            }
            return [
            {
                value: phoneNumber,
                info: [
                {
                    message:
                    'We have automatically transformed your input into the correct format by converting "00" to "+".',
                    level: "info",
                },
                ],
            },
            index,
            ];
        });
        },
    }}
    >
    Import data
    </NuvoImporter>""")
    prompt = prompt.partial(cols= cols)
    llm_chain = LLMChain(llm=openai, prompt=prompt)
    output = llm_chain.invoke(actions)
    return output["text"]

In [6]:
prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI(model = "gpt-4-0125-preview", api_key="sk-c", temperature = 0)


In [7]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [8]:
# agent_executor = AgentExecutor(agent = agent, tools=[get_columns, generate_cleaning_code], handle_parsing_errors=True, verbose=True, max_iterations=3)
# columns = agent_executor.invoke({"input": """get the list of columns from crm_demo.xlsx and then pass all the columns to the cleaning_function along with the following actions to perform:
#                                 1. City names to capital\n2. Names to capital 
#                              Output just the code without any explanations"""})['output']

from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import AgentType

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[get_columns, generate_cleaning_code],
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    memory=conversational_memory,
)

d:\Programs\minconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
actions = "1. City names to capital\n2. Names to capital" 
response = agent(
    f"get the list of columns from crm_demo.xlsx and then pass all the columns to the cleaning_function along with the following actions to perform: {actions} Output just the code without any explanations"
)


d:\Programs\minconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_columns",
  "action_input": {
    "file_name": "crm_demo.xlsx"
  }
}
```
Observation: ['Company Identification Number ', 'Organisation', 'Mail', 'Number', 'Website', 'Street', 'City', 'Country', 'Amount', 'Department', 'Expenditures', 'Closing Date', 'First Name', 'Last Name']
Thought:Action:
```
{
  "action": "generate_cleaning_code",
  "action_input": {
    "actions": "City names to capital, Names to capital",
    "cols": ["Company Identification Number ", "Organisation", "Mail", "Number", "Website", "Street", "City", "Country", "Amount", "Department", "Expenditures", "Closing Date", "First Name", "Last Name"]
  }
}
```
Observation: ```typescript
<NuvoImporter
  licenseKey="Your License Key"
  settings={{
    developerMode: true,
    identifier: "customer_data",
    columns: [
      {
        label: "Company Identification Number",
        key: "company_identification_number",
        columnType: "string",
      }

In [10]:
response['output']

'<NuvoImporter\n  licenseKey="Your License Key"\n  settings={{\n    developerMode: true,\n    identifier: "customer_data",\n    columns: [\n      {\n        label: "Company Identification Number",\n        key: "company_identification_number",\n        columnType: "string",\n      },\n      {\n        label: "Organisation",\n        key: "organisation",\n        columnType: "string",\n      },\n      {\n        label: "Mail",\n        key: "mail",\n        columnType: "string",\n      },\n      {\n        label: "Number",\n        key: "number",\n        columnType: "string",\n      },\n      {\n        label: "Website",\n        key: "website",\n        columnType: "string",\n      },\n      {\n        label: "Street",\n        key: "street",\n        columnType: "string",\n      },\n      {\n        label: "City",\n        key: "city",\n        columnType: "string",\n      },\n      {\n        label: "Country",\n        key: "country",\n        columnType: "string",\n      },\n      